# Unsup3D


In [ ]:
# Setup 
!git clone https://github.com/elliottwu/unsup3d.git

Cloning into 'unsup3d'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 180 (delta 7), reused 15 (delta 6), pack-reused 160
Receiving objects: 100% (180/180), 2.33 MiB | 3.27 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [ ]:
# Installations
!pip install scikit-image
!pip install matplotlib
!pip install opencv
!pip install moviepy
!pip install pyyaml
!pip install tensorboardX
!pip install torch==1.2.0 torchvision==0.4.0 mmcv==1.2.7 
!pip install neural_renderer_pytorch
!pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 29.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 748.9 MB 658 bytes/s 
     |████████████████████████████████| 8.8 MB

In [ ]:
# Download pretrained model
%cd /content/unsup3d/
!curl -o pretrained_synface.zip "https://www.robots.ox.ac.uk/~vgg/research/unsup3d/data/pretrained_synface.zip" && unzip pretrained_synface.zip

/content/unsup3d
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  383M  100  383M    0     0  12.0M      0  0:00:31  0:00:31 --:--:-- 13.9M
Archive:  pretrained_synface.zip
   creating: pretrained_synface/
  inflating: pretrained_synface/checkpoint030.pth  


## Demo #1

In [ ]:
# Demo
# --gpu: enable GPU
# --detect_human_face: enable automatic human face detection and cropping using MTCNN provided in facenet-pytorch. 
# --render_video: render 3D animations using neural_renderer (GPU is required)
%cd /content/unsup3d/
results_path = '/content/results' # @type={"string"}
!python -m demo.demo --input demo/images/human_face --result {results_path} --checkpoint pretrained_synface/checkpoint030.pth --gpu
print("The results can be found under", results_path)

/content/unsup3d
Loading checkpoint from pretrained_synface/checkpoint030.pth
Processing demo/images/human_face/001_face.png
Saving results to /content/results/001_face
Processing demo/images/human_face/002_face.png
Saving results to /content/results/002_face
Processing demo/images/human_face/003_face.png
Saving results to /content/results/003_face
Processing demo/images/human_face/004_face.png
Saving results to /content/results/004_face
Processing demo/images/human_face/005_face.png
Saving results to /content/results/005_face
Processing demo/images/human_face/006_face.png
Saving results to /content/results/006_face
Processing demo/images/human_face/007_face.png
Saving results to /content/results/007_face
Processing demo/images/human_face/008_face.png
Saving results to /content/results/008_face
Processing demo/images/human_face/009_face.png
Saving results to /content/results/009_face
Processing demo/images/human_face/010_face.png
Saving results to /content/results/010_face
Processing d

## Demo #2
Face and depth reconstruction from video input

In [ ]:
# Demo
# --gpu: enable GPU
# --detect_human_face: enable automatic human face detection and cropping using MTCNN provided in facenet-pytorch. 
# --render_video: render 3D animations using neural_renderer (GPU is required)
%cd /content/unsup3d/
results_path = '/content/results' # @type={"string"}
!python -m demo.demo --input demo/images/human_face --result {results_path} --checkpoint pretrained_synface/checkpoint030.pth --gpu \
--render_video
print("The results can be found under", results_path)

/content/unsup3d
Loading checkpoint from pretrained_synface/checkpoint030.pth
Processing demo/images/human_face/001_face.png
Rendering video animations
Saving results to /content/results/001_face
Processing demo/images/human_face/002_face.png
Rendering video animations
Saving results to /content/results/002_face
Processing demo/images/human_face/003_face.png
Rendering video animations
Saving results to /content/results/003_face
Processing demo/images/human_face/004_face.png
Rendering video animations
Saving results to /content/results/004_face
Processing demo/images/human_face/005_face.png
Rendering video animations
Saving results to /content/results/005_face
Processing demo/images/human_face/006_face.png
Rendering video animations
Saving results to /content/results/006_face
Processing demo/images/human_face/007_face.png
Rendering video animations
Saving results to /content/results/007_face
Processing demo/images/human_face/008_face.png
Rendering video animations
Saving results to /con

## Play Around

In [ ]:
# Connect to Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@markdown Define new testing function
%%writefile '/content/unsup3d/demo/demo_new.py' 

import argparse
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from .utils import *


EPS = 1e-7


class Demo_new():
    def __init__(self, args):
        ## configs
        self.device = 'cuda:0' if args.gpu else 'cpu'
        self.checkpoint_path = args.checkpoint
        self.detect_human_face = False
        self.render_video = args.render_video
        self.output_size = args.output_size
        self.image_size = 64
        self.min_depth = 0.9
        self.max_depth = 1.1
        self.border_depth = 1.00
        self.xyz_rotation_range = 60
        self.xy_translation_range = 0.1
        self.z_translation_range = 0
        self.fov = 10 # in degrees
        

        self.depth_rescaler = lambda d : (1+d)/2 *self.max_depth + (1-d)/2 *self.min_depth  # (-1,1) => (min_depth,max_depth)
        self.depth_inv_rescaler = lambda d :  (d-self.max_depth) / (self.min_depth-self.max_depth)  # (min_depth,max_depth) => (0,1)

        fx = (self.image_size-1)/2/(np.tan(self.fov/2 *np.pi/180))
        fy = (self.image_size-1)/2/(np.tan(self.fov/2 *np.pi/180))
        cx = (self.image_size-1)/2
        cy = (self.image_size-1)/2
        K = [[fx, 0., cx],
             [0., fy, cy],
             [0., 0., 1.]]
        K = torch.FloatTensor(K).to(self.device)
        self.inv_K = torch.inverse(K).unsqueeze(0)
        self.K = K.unsqueeze(0)

        ## NN models
        self.netD = EDDeconv(cin=3, cout=1, nf=64, zdim=256, activation=None)
        self.netA = EDDeconv(cin=3, cout=3, nf=64, zdim=256)
        self.netL = Encoder(cin=3, cout=4, nf=32)
        self.netV = Encoder(cin=3, cout=6, nf=32)

        self.netD = self.netD.to(self.device)
        self.netA = self.netA.to(self.device)
        self.netL = self.netL.to(self.device)
        self.netV = self.netV.to(self.device)
        self.load_checkpoint()

        self.netD.eval()
        self.netA.eval()
        self.netL.eval()
        self.netV.eval()

        ## face detecter
        if self.detect_human_face:
            from facenet_pytorch import MTCNN
            self.face_detector = MTCNN(select_largest=True, device=self.device)

        ## renderer
        if self.render_video:
            from unsup3d.renderer import Renderer
            assert 'cuda' in self.device, 'A GPU device is required for rendering because the neural_renderer only has GPU implementation.'
            cfgs = {
                'device': self.device,
                'image_size': self.output_size,
                'min_depth': self.min_depth,
                'max_depth': self.max_depth,
                'fov': self.fov,
            }
            self.renderer = Renderer(cfgs)

    def load_checkpoint(self):
        print(f"Loading checkpoint from {self.checkpoint_path}")
        cp = torch.load(self.checkpoint_path, map_location=self.device)
        self.netD.load_state_dict(cp['netD'])
        self.netA.load_state_dict(cp['netA'])
        self.netL.load_state_dict(cp['netL'])
        self.netV.load_state_dict(cp['netV'])

    def depth_to_3d_grid(self, depth, inv_K=None):
        if inv_K is None:
            inv_K = self.inv_K
        b, h, w = depth.shape
        grid_2d = get_grid(b, h, w, normalize=False).to(depth.device)  # Nxhxwx2
        depth = depth.unsqueeze(-1)
        grid_3d = torch.cat((grid_2d, torch.ones_like(depth)), dim=3)
        grid_3d = grid_3d.matmul(inv_K.transpose(2,1)) * depth
        return grid_3d

    def get_normal_from_depth(self, depth):
        b, h, w = depth.shape
        grid_3d = self.depth_to_3d_grid(depth)

        tu = grid_3d[:,1:-1,2:] - grid_3d[:,1:-1,:-2]
        tv = grid_3d[:,2:,1:-1] - grid_3d[:,:-2,1:-1]
        normal = tu.cross(tv, dim=3)

        zero = normal.new_tensor([0,0,1])
        normal = torch.cat([zero.repeat(b,h-2,1,1), normal, zero.repeat(b,h-2,1,1)], 2)
        normal = torch.cat([zero.repeat(b,1,w,1), normal, zero.repeat(b,1,w,1)], 1)
        normal = normal / (((normal**2).sum(3, keepdim=True))**0.5 + EPS)
        return normal

    def detect_face(self, im):
        print("Detecting face using MTCNN face detector")
        try:
            bboxes, prob = self.face_detector.detect(im)
            w0, h0, w1, h1 = bboxes[0]
        except:
            print("Could not detect faces in the image")
            return None

        hc, wc = (h0+h1)/2, (w0+w1)/2
        crop = int(((h1-h0) + (w1-w0)) /2/2 *1.1)
        im = np.pad(im, ((crop,crop),(crop,crop),(0,0)), mode='edge')  # allow cropping outside by replicating borders
        h0 = int(hc-crop+crop + crop*0.15)
        w0 = int(wc-crop+crop)
        return im[h0:h0+crop*2, w0:w0+crop*2]

    def run(self, pil_im):
        im = np.uint8(pil_im)

        ## face detection
        if self.detect_human_face:
            im = self.detect_face(im)
            if im is None:
                return -1

        h, w, _ = im.shape
        im = torch.FloatTensor(im /255.).permute(2,0,1).unsqueeze(0)
        # resize to 128 first if too large, to avoid bilinear downsampling artifacts
        if h > self.image_size*4 and w > self.image_size*4:
            im = nn.functional.interpolate(im, (self.image_size*2, self.image_size*2), mode='bilinear', align_corners=False)
        im = nn.functional.interpolate(im, (self.image_size, self.image_size), mode='bilinear', align_corners=False)

        with torch.no_grad():
            self.input_im = im.to(self.device) *2.-1.
            b, c, h, w = self.input_im.shape

            ## predict canonical depth
            self.canon_depth_raw = self.netD(self.input_im).squeeze(1)  # BxHxW
            self.canon_depth = self.canon_depth_raw - self.canon_depth_raw.view(b,-1).mean(1).view(b,1,1)
            self.canon_depth = self.canon_depth.tanh()
            self.canon_depth = self.depth_rescaler(self.canon_depth)

            from PIL import Image
            

            ## clamp border depth
            depth_border = torch.zeros(1,h,w-4).to(self.input_im.device)
            depth_border = nn.functional.pad(depth_border, (2,2), mode='constant', value=1)
            self.canon_depth = self.canon_depth*(1-depth_border) + depth_border *self.border_depth



            
            
            
            ## predict canonical albedo
            self.canon_albedo = self.netA(self.input_im)  # Bx3xHxW

            ## predict lighting
            canon_light = self.netL(self.input_im)  # Bx4
            self.canon_light_a = canon_light[:,:1] /2+0.5  # ambience term
            self.canon_light_b = canon_light[:,1:2] /2+0.5  # diffuse term
            canon_light_dxy = canon_light[:,2:]
            self.canon_light_d = torch.cat([canon_light_dxy, torch.ones(b,1).to(self.input_im.device)], 1)
            self.canon_light_d = self.canon_light_d / ((self.canon_light_d**2).sum(1, keepdim=True))**0.5  # diffuse light direction

            ## shading
            self.canon_normal = self.get_normal_from_depth(self.canon_depth)
            self.canon_diffuse_shading = (self.canon_normal * self.canon_light_d.view(-1,1,1,3)).sum(3).clamp(min=0).unsqueeze(1)
            canon_shading = self.canon_light_a.view(-1,1,1,1) + self.canon_light_b.view(-1,1,1,1)*self.canon_diffuse_shading
            self.canon_im = (self.canon_albedo/2+0.5) * canon_shading *2-1

            ## predict viewpoint transformation
            self.view = self.netV(self.input_im)
            self.view = torch.cat([
                self.view[:,:3] *np.pi/180 *self.xyz_rotation_range,
                self.view[:,3:5] *self.xy_translation_range,
                self.view[:,5:] *self.z_translation_range], 1)



            '''New shit'''
            from unsup3d.renderer import Renderer
            assert 'cuda' in self.device, 'A GPU device is required for rendering because the neural_renderer only has GPU implementation.'
            cfgs = {
                'device': self.device,
                'image_size': self.output_size,
                'min_depth': self.min_depth,
                'max_depth': self.max_depth,
                'fov': self.fov,
            }
            self.renderer = Renderer(cfgs)
            self.renderer.set_transform_matrices(self.view)
            wrapped_depth = self.renderer.warp_canon_depth(self.canon_depth)
            save_image('/content/depth.png', self.depth_inv_rescaler(wrapped_depth)[0].repeat(3,1,1), 'canonical_depth')

            ## export to obj strings
            vertices = self.depth_to_3d_grid(self.canon_depth)  # BxHxWx3
            self.objs, self.mtls = export_to_obj_string(vertices, self.canon_normal)

            ## resize to output size
            self.canon_depth = nn.functional.interpolate(self.canon_depth.unsqueeze(1), (self.output_size, self.output_size), mode='bilinear', align_corners=False).squeeze(1)
            self.canon_normal = nn.functional.interpolate(self.canon_normal.permute(0,3,1,2), (self.output_size, self.output_size), mode='bilinear', align_corners=False).permute(0,2,3,1)
            self.canon_normal = self.canon_normal / (self.canon_normal**2).sum(3, keepdim=True)**0.5
            self.canon_diffuse_shading = nn.functional.interpolate(self.canon_diffuse_shading, (self.output_size, self.output_size), mode='bilinear', align_corners=False)
            self.canon_albedo = nn.functional.interpolate(self.canon_albedo, (self.output_size, self.output_size), mode='bilinear', align_corners=False)
            self.canon_im = nn.functional.interpolate(self.canon_im, (self.output_size, self.output_size), mode='bilinear', align_corners=False)

            if self.render_video:
                self.render_animation()

    def render_animation(self):
        print(f"Rendering video animations")
        b, h, w = self.canon_depth.shape

        ## morph from target view to canonical
        morph_frames = 15
        view_zero = torch.FloatTensor([0.15*np.pi/180*60, 0,0,0,0,0]).to(self.canon_depth.device)
        morph_s = torch.linspace(0, 1, morph_frames).to(self.canon_depth.device)
        view_morph = morph_s.view(-1,1,1) * view_zero.view(1,1,-1) + (1-morph_s.view(-1,1,1)) * self.view.unsqueeze(0)  # TxBx6

        ## yaw from canonical to both sides
        yaw_frames = 80
        yaw_rotations = np.linspace(-np.pi/2, np.pi/2, yaw_frames)
        # yaw_rotations = np.concatenate([yaw_rotations[40:], yaw_rotations[::-1], yaw_rotations[:40]], 0)

        ## whole rotation sequence
        view_after = torch.cat([view_morph, view_zero.repeat(yaw_frames, b, 1)], 0)
        yaw_rotations = np.concatenate([np.zeros(morph_frames), yaw_rotations], 0)

        def rearrange_frames(frames):
            morph_seq = frames[:, :morph_frames]
            yaw_seq = frames[:, morph_frames:]
            out_seq = torch.cat([
                morph_seq[:,:1].repeat(1,5,1,1,1),
                morph_seq,
                morph_seq[:,-1:].repeat(1,5,1,1,1),
                yaw_seq[:, yaw_frames//2:],
                yaw_seq.flip(1),
                yaw_seq[:, :yaw_frames//2],
                morph_seq[:,-1:].repeat(1,5,1,1,1),
                morph_seq.flip(1),
                morph_seq[:,:1].repeat(1,5,1,1,1),
            ], 1)
            return out_seq

        ## textureless shape
        front_light = torch.FloatTensor([0,0,1]).to(self.canon_depth.device)
        canon_shape_im = (self.canon_normal * front_light.view(1,1,1,3)).sum(3).clamp(min=0).unsqueeze(1)
        canon_shape_im = canon_shape_im.repeat(1,3,1,1) *0.7
        shape_animation = self.renderer.render_yaw(canon_shape_im, self.canon_depth, v_after=view_after, rotations=yaw_rotations)  # BxTxCxHxW
        self.shape_animation = rearrange_frames(shape_animation)

        ## normal map
        canon_normal_im = self.canon_normal.permute(0,3,1,2) /2+0.5
        normal_animation = self.renderer.render_yaw(canon_normal_im, self.canon_depth, v_after=view_after, rotations=yaw_rotations)  # BxTxCxHxW
        self.normal_animation = rearrange_frames(normal_animation)

        ## textured
        texture_animation = self.renderer.render_yaw(self.canon_im /2+0.5, self.canon_depth, v_after=view_after, rotations=yaw_rotations)  # BxTxCxHxW
        self.texture_animation = rearrange_frames(texture_animation)

    def save_results(self, save_dir):
        print(f"Saving results to {save_dir}")
        save_image(save_dir, self.input_im[0]/2+0.5, 'input_image')
        save_image(save_dir, self.depth_inv_rescaler(self.canon_depth)[0].repeat(3,1,1), 'canonical_depth')
        save_image(save_dir, self.canon_normal[0].permute(2,0,1)/2+0.5, 'canonical_normal')
        save_image(save_dir, self.canon_diffuse_shading[0].repeat(3,1,1), 'canonical_diffuse_shading')
        save_image(save_dir, self.canon_albedo[0]/2+0.5, 'canonical_albedo')
        save_image(save_dir, self.canon_im[0].clamp(-1,1)/2+0.5, 'canonical_image')
        save_image(save_dir, self.depth_inv_rescaler(self.canon_depth)[0].repeat(3,1,1), 'wraped_depth')

        with open(os.path.join(save_dir, 'result.mtl'), "w") as f:
            f.write(self.mtls[0].replace('$TXTFILE', './canonical_image.png'))
        with open(os.path.join(save_dir, 'result.obj'), "w") as f:
            f.write(self.objs[0].replace('$MTLFILE', './result.mtl'))

        if self.render_video:
            save_video(save_dir, self.shape_animation[0], 'shape_animation')
            save_video(save_dir, self.normal_animation[0], 'normal_animation')
            save_video(save_dir, self.texture_animation[0], 'texture_animation')


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Demo configurations.')
    parser.add_argument('--input', default='./demo/images/human_face', type=str, help='Path to the directory containing input images')
    parser.add_argument('--result', default='./demo/results/human_face', type=str, help='Path to the directory for saving results')
    parser.add_argument('--checkpoint', default='./pretrained/pretrained_celeba/checkpoint030.pth', type=str, help='Path to the checkpoint file')
    parser.add_argument('--output_size', default=128, type=int, help='Output image size')
    parser.add_argument('--gpu', default=False, action='store_true', help='Enable GPU')
    parser.add_argument('--detect_human_face', default=False, action='store_true', help='Enable automatic human face detection. This does not detect cat faces.')
    parser.add_argument('--render_video', default=False, action='store_true', help='Render 3D animations to video')
    args = parser.parse_args()

    input_dir = args.input
    result_dir = args.result
    model = Demo_new(args)
    im_list = [os.path.join(input_dir, f) for f in sorted(os.listdir(input_dir)) if is_image_file(f)]

    for im_path in im_list:
        print(f"Processing {im_path}")
        pil_im = Image.open(im_path).convert('RGB')
        result_code = model.run(pil_im)
        if result_code == -1:
            print(f"Failed! Skipping {im_path}")
            continue

        save_dir = os.path.join(result_dir, os.path.splitext(os.path.basename(im_path))[0])
        model.save_results(save_dir)


Overwriting /content/unsup3d/demo/demo_new.py


### Evaluation Set

In [ ]:
# Specific image
%cd /content/unsup3d/
#results_path = '/content/drive/MyDrive/experiments/unsup3d_vs_attention_evaluation/' # @type={"string"}
results_path = '/content/check/'
input_folder = '/content/drive/MyDrive/datasets/evaluation_set/'

!python -m demo.demo_new --input {input_folder} --result {results_path} --checkpoint pretrained_synface/checkpoint030.pth --gpu
print("The results can be found under", results_path)

/content/unsup3d
Loading checkpoint from pretrained_synface/checkpoint030.pth
Processing /content/drive/MyDrive/datasets/evaluation_set/01523.png
Saving results to /content/check/01523
Processing /content/drive/MyDrive/datasets/evaluation_set/01537.png
Saving results to /content/check/01537
Processing /content/drive/MyDrive/datasets/evaluation_set/01574.png
Saving results to /content/check/01574
Processing /content/drive/MyDrive/datasets/evaluation_set/48000.png
Saving results to /content/check/48000
Processing /content/drive/MyDrive/datasets/evaluation_set/48002.png
Saving results to /content/check/48002
Processing /content/drive/MyDrive/datasets/evaluation_set/48004.png
Saving results to /content/check/48004
Processing /content/drive/MyDrive/datasets/evaluation_set/48008.png
Saving results to /content/check/48008
Processing /content/drive/MyDrive/datasets/evaluation_set/cristiano2.png.jpeg
Saving results to /content/check/cristiano2.png
The results can be found under /content/check/


### Real Set

In [ ]:
# Predict and save results for prosopo dataset
root_path = '/content/drive/MyDrive/datasets/prosopo_new/input/'
output_path = '/content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/'
!python -m demo.demo --input {root_path} --result {output_path} --checkpoint pretrained_synface/checkpoint030.pth --gpu --output_size 128 # default depth size is 128
# set max depth to 1 and min depth to -1

Loading checkpoint from pretrained_synface/checkpoint030.pth
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00002.png
Saving results to /content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/00002
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00003.png
Saving results to /content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/00003
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00004.png
Saving results to /content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/00004
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00005.png
Saving results to /content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/00005
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00006.png
Saving results to /content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/00006
Processing /content/drive/MyDrive/datasets/prosopo_new/input/00007.png
Saving results to /content/drive/MyDrive/eg3d/experim

### Synthetic set

In [ ]:
# Predict and save results for synthetic dataset
root_path = '/content/drive/MyDrive/datasets/prosopo_new/input/'
output_path = '/content/drive/MyDrive/eg3d/experiments/unsup3d_prosopo_predictions/'
!python -m demo.demo --input {root_path} --result {output_path} --checkpoint pretrained_synface/checkpoint030.pth --gpu --output_size 128 # default depth size is 128
# set max depth to 1 and min depth to -1

In [ ]:
#@markdown Calculate l1-loss between 2 depthmaps
def l1_loss(pred_path='/content/pred_depth.png', gt_path='/content/gt_depth.png'):
  pred = np.array(Image.open(pred_path))
  gt = np.array(Image.open(gt_path))
  loss = l1(pred, gt)
  return loss


In [ ]:
# Evaluate results on l1-loss gt depth loss
from PIL import Image
import numpy as np
import os
gt_path = '/content/drive/MyDrive/datasets/prosopo_new/depth128x128'
root_path = '/content/drive/MyDrive/experiments/eg3d/' # path for depth predictions
for example in os.listdir(root_path):
  example_path = os.path.join(root_path, example)
  pred_path = os.path.join(example_path, 'canonical_depth.png')
  gt_path = os.path.join(gt_path, f'{example}.png')
  sample_loss = l1_loss(pred_path=pred_path, gt_path=gt_path)
  print(f'Loss  for {example} is: {sample_loss}')


SyntaxError: ignored